In [3]:
!pip install sentence_transformers pinecone-client datasets seaborn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/

In [4]:
!pip install --upgrade torch torchvision

In [4]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";
os.environ["CUDA_LAUNCH_BLOCKING"]="1";


In [5]:
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,TrainingArguments
)

In [6]:


class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [7]:
# load the dataset and convert to pandas dataframe
df_tain = load_dataset("fhamborg/news_sentiment_newsmtsc",split="train").to_pandas()

In [8]:
df_validation = load_dataset( "fhamborg/news_sentiment_newsmtsc",split="validation").to_pandas()

In [9]:
df_test = load_dataset("fhamborg/news_sentiment_newsmtsc",split="test").to_pandas()

In [57]:
df_test["sentence"][0]

'Though we do not know what other items Seth may have had in his possession, his watch, phone, wallet and necklace were not stolen.'

In [58]:
df_test

,mention,polarity,from,to,sentence,id
0,Seth,0,39,43,Though we do not know what other items Seth ma...,polusa_v1_4307505_-1_11_Seth_39_43
1,He,1,0,2,He won the Academy Award for Best Supporting A...,allsides_4276_1784_7_Beloved actor Robin Willi...
2,""" Powell",1,44,52,"""Yup, the whole birther movement was racist,"" ...",allsides_1810_737_6_Powell_44_52
3,Ross,0,0,4,Ross's appointment will require a Senate confi...,allsides_1593_643_5_Wilbur Ross_0_4
4,Andrews,0,27,34,Trustees Simone Boutet and Andrews voted no ea...,polusa_v1_59535720_-1_32_Trustee Deno Andrews_...
...,...,...,...,...,...,...
798,Poe,0,91,94,To have her now say she doesn't support our ri...,allsides_4841_2036_10_Poe_91_94
799,Obama,1,98,103,But some observers suggested it was tone-deaf ...,allsides_2495_1015_21_Obama_98_103
800,Cox,0,95,98,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Sean Cox_95_98
801,Schmidt,-1,104,111,“It is my opinion that you are a key conspirat...,polusa_v1_39150634_-1_5_Schmidt_104_111


In [11]:
clean_1_test = df_test.groupby('polarity', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
clean_1_test = clean_1_test.sample(frac=1, random_state=42)
clean_1_test.dropna(subset=['sentence'], inplace=True)
clean_1_test

,mention,polarity,from,to,sentence,id
696,He,0,0,2,He has Bill Clinton.”,allsides_6109_2624_24_Romney_0_2
714,Kavanaugh,-1,13,22,A 1999 op-ed Kavanaugh wrote for the Wall Stre...,polusa_v1_4283918_-1_10_Judge Brett Kavanaugh_...
143,Pelosi,1,0,6,Pelosi was undisputedly correct about one thin...,allsides_4477_1871_25_House Minority Leader Na...
316,Mueller,0,150,157,The Associated Press reported that the GSA tur...,allsides_187_77_6_Mueller_150_157
470,about 21 years old Linda Mustion,0,43,75,While most of the soldiers on the wall are abo...,polusa_v1_60289022_-1_26_about 21 years old Li...
...,...,...,...,...,...,...
124,he,0,61,63,“We organized demonstrations in front of their...,polusa_v1_4077736_-1_60_Mr. Asgari_62_64
100,she,1,133,136,"Republicans invested millions behind Kim, a ri...",polusa_v1_3905270_-1_17_Democrat Katie Porter_...
366,Trump,-1,31,36,His performance last night was Trump being Tru...,allsides_2983_1220_61_Donald Trump's_31_36
569,Ferguson,0,17,25,When asked about Ferguson’s finding that Escam...,polusa_v1_59556386_-1_33_Ferguson_17_25


In [12]:
x_train=df_tain["sentence"]
y_train=df_tain["polarity"]

In [13]:
x_valid=df_validation["sentence"]
y_valid=df_validation["polarity"]

In [14]:
x_test=clean_1_test["sentence"]
y_test=clean_1_test["polarity"]

In [16]:
# model_id = "cardiffnlp/twitter-roberta-base-sentiment"
# model_id ="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
model_id ="distilbert-base-uncased"
# model_id ="distilbert-base-uncased-finetuned-sst-2-english"

# model_id ="distilbert-base-uncased-distilled-squad"

# model_id ="Davlan/distilbert-base-multilingual-cased-ner-hrl"
# model_id ="xlnet-base-cased"
# model_id ="t5-small"

# load the model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=3
                                                          #  , ignore_mismatched_sizes=True
                                                           )
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
train_encoding = tokenizer(x_train.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
valid_encoding = tokenizer(x_valid.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
test_encoding = tokenizer(x_test.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")


In [18]:
# train_encoding=tokenizer(x_train.tolist(),padding=True,truncation=True)
# valid_encoding=tokenizer(x_valid.tolist(),padding=True,truncation=True)
# test_encoding=tokenizer(x_test.tolist(),padding=True,truncation=True)


In [19]:
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)
y_test = y_test.astype(int)


In [20]:
y_train

0       0
1      -1
2      -1
3      -1
4      -1
       ..
8734   -1
8735    0
8736    0
8737    1
8738   -1
Name: polarity, Length: 8739, dtype: int64

In [21]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the target labels and transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)
y_test_encoded = label_encoder.transform(y_test)



In [22]:
y_train.unique()

array([ 0, -1,  1])

In [41]:
{"1":0,"0":-1,"2":1}

,mention,polarity,from,to,sentence,id
0,Winner,0,0,6,Winner wrote that she had a 30-minute private ...,allsides_1000_401_25_Reality Leigh Winner_0_6
1,She,-1,0,3,She also recently referred to President Trump ...,allsides_1000_401_3_Reality Leigh Winner_0_3
2,President Trump,-1,30,45,She also recently referred to President Trump ...,allsides_1000_401_3_’ Trump_30_45
3,Hillary Clinton,-1,0,15,Hillary Clinton blamed the Democratic National...,allsides_1018_408_1_Hillary Clinton_2_17
4,Facebook,-1,58,66,Hillary Clinton blamed the Democratic National...,allsides_1018_408_1_Facebook_60_68
...,...,...,...,...,...,...
8734,Acosta,-1,14,20,"In announcing Acosta's suspension, Sanders sai...",polusa_v1_59576418_-1_41_CNN correspondent Jim...
8735,Cassidy,0,0,7,Cassidy has sought to defend the provision.,allsides_569_226_44_Graham-Cassidy_0_7
8736,Trump,0,0,5,Trump returned to the United States on Tuesday...,polusa_v1_4359001_-1_13_Donald Trump_1_6
8737,Clinton,1,104,111,A Wall Street Journal/NBC poll taken after the...,allsides_1732_705_20_Hillary Clinton_104_111


In [23]:
y_train_encoded

array([1, 0, 0, ..., 1, 2, 0])

In [24]:
train_dataset=IMDbDataset(train_encoding,y_train_encoded)
valid_dataset=IMDbDataset(valid_encoding,y_valid_encoded)
test_dataset=IMDbDataset(test_encoding,y_test_encoded)


In [25]:
train_dataset

In [26]:
training_args = TrainingArguments(
    output_dir='./results',  # Fix the space before the directory path
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
    # compute_metrics=compute_metrics,
)

trainer.train()

In [28]:
# output_dir="saved"
# tokenizer.save_pretrained(output_dir)

# model.save_pretrained(output_dir)

In [34]:
trainer.save_model()


trainer.save_state()


checkpoint_name = "test-trainer-deepspeed"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"



trainer.save_model(output_dir=final_model_path)

In [29]:
# fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
# tokenizer_new = AutoTokenizer.from_pretrained(output_dir)


In [36]:

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(final_model_path)
# tokenizer_new = AutoTokenizer.from_pretrained(final_model_path)

In [37]:
inputs = tokenizer(x_test.tolist(),return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [ ]:
import torch.nn.functional as F
with torch.no_grad():
    outputs =fine_tuned_model(**inputs)
    print (outputs)
    prediction=F.softmax(outputs.logits, dim=1 )
    print (prediction)

    labels= torch.argmax(prediction, dim=1 )
    print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

In [47]:
label_mapping = {"LABEL_1": 0, "LABEL_0": -1, "LABEL_2": 1}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [ ]:
classification_nnn

In [56]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test, classification_nnn))

              precision    recall  f1-score   support

          -1       0.66      0.78      0.72        50
           0       0.73      0.74      0.73        50
           1       0.80      0.64      0.71        50

    accuracy                           0.72       150
   macro avg       0.73      0.72      0.72       150
weighted avg       0.73      0.72      0.72       150



# Garbage

# strat_first

In [ ]:
from datasets import load_dataset
import os
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,TrainingArguments
)

In [2]:
model_id = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"

# load the model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(
    model_id
    # ,num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
# load the dataset and convert to pandas dataframe
df = load_dataset(
    "fhamborg/news_sentiment_newsmtsc",
    split="train"
)

In [4]:
df

Dataset({
    features: ['mention', 'polarity', 'from', 'to', 'sentence', 'id'],
    num_rows: 8739
})

In [5]:
###### modify L78 to disable fast tokenizer
default=False,

###### update dataset map part at L313
dataset = df.map(tokenizer, input_columns="sentence", fn_kwargs={"padding": "max_length", "truncation": True, "max_length": 512})

In [6]:
dataset["sentence"][15]

'It was a familiar morning for Mr. Trump on his favored medium — and for a nation that had, for a little more than a week, gone without the president’s stream-of-consciousness missives.'

In [13]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

# ...

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=tokenizer.pad_token_id,  # Ensure label padding
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  # Use the data collator for sequence-to-sequence tasks
    train_dataset=dataset,
)

trainer.train()


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: ignored

In [9]:
local_training_root = "./"
checkpoint_name = "test-trainer"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)

training_args = TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,  # default number of epochs to train is 3
    per_device_train_batch_size=16,
    optim="adamw_torch",
    report_to=["tensorboard"]
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=tokenized_dataset["test"],
    # compute_metrics=compute_metrics,
)

trainer.train()

ValueError: ignored

In [ ]:
###### modify L78 to disable fast tokenizer
default=False,

###### update dataset map part at L313
dataset = df.map(tokenizer, input_columns="sentence", fn_kwargs={"padding": "max_length", "truncation": True, "max_length": 512})

###### add following lines to L213
del model
print(f"Manually deleted Teacher model, free some memory for student model.")

###### add following lines to L337
trainer.push_to_hub()
tokenizer.push_to_hub("distilbert-base-multilingual-cased-sentiments-student")


# strat

In [1]:
!pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s

In [2]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer


In [21]:
dataset = load_dataset("fhamborg/news_sentiment_newsmtsc")

In [22]:

dataset_train = dataset["train"].to_pandas()
dataset_train


,mention,polarity,from,to,sentence,id
0,Winner,0,0,6,Winner wrote that she had a 30-minute private ...,allsides_1000_401_25_Reality Leigh Winner_0_6
1,She,-1,0,3,She also recently referred to President Trump ...,allsides_1000_401_3_Reality Leigh Winner_0_3
2,President Trump,-1,30,45,She also recently referred to President Trump ...,allsides_1000_401_3_’ Trump_30_45
3,Hillary Clinton,-1,0,15,Hillary Clinton blamed the Democratic National...,allsides_1018_408_1_Hillary Clinton_2_17
4,Facebook,-1,58,66,Hillary Clinton blamed the Democratic National...,allsides_1018_408_1_Facebook_60_68
...,...,...,...,...,...,...
8734,Acosta,-1,14,20,"In announcing Acosta's suspension, Sanders sai...",polusa_v1_59576418_-1_41_CNN correspondent Jim...
8735,Cassidy,0,0,7,Cassidy has sought to defend the provision.,allsides_569_226_44_Graham-Cassidy_0_7
8736,Trump,0,0,5,Trump returned to the United States on Tuesday...,polusa_v1_4359001_-1_13_Donald Trump_1_6
8737,Clinton,1,104,111,A Wall Street Journal/NBC poll taken after the...,allsides_1732_705_20_Hillary Clinton_104_111


In [24]:
train_ds=dataset_train[['polarity','sentence']]
train_ds

,polarity,sentence
0,0,Winner wrote that she had a 30-minute private ...
1,-1,She also recently referred to President Trump ...
2,-1,She also recently referred to President Trump ...
3,-1,Hillary Clinton blamed the Democratic National...
4,-1,Hillary Clinton blamed the Democratic National...
...,...,...
8734,-1,"In announcing Acosta's suspension, Sanders sai..."
8735,0,Cassidy has sought to defend the provision.
8736,0,Trump returned to the United States on Tuesday...
8737,1,A Wall Street Journal/NBC poll taken after the...


In [25]:
train_ds.rename(columns={'polarity': 'label', 'sentence': 'text'}, inplace=True)
train_ds

<ipython-input-25-e6e005f18a4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ds.rename(columns={'polarity': 'label', 'sentence': 'text'}, inplace=True)


,label,text
0,0,Winner wrote that she had a 30-minute private ...
1,-1,She also recently referred to President Trump ...
2,-1,She also recently referred to President Trump ...
3,-1,Hillary Clinton blamed the Democratic National...
4,-1,Hillary Clinton blamed the Democratic National...
...,...,...
8734,-1,"In announcing Acosta's suspension, Sanders sai..."
8735,0,Cassidy has sought to defend the provision.
8736,0,Trump returned to the United States on Tuesday...
8737,1,A Wall Street Journal/NBC poll taken after the...


In [26]:
# Select N examples per class (8 in this case)
# train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

In [27]:
from datasets import Dataset, load_metric
dataset_TEMP = Dataset.from_pandas(train_ds)


dataset_TEMP

Dataset({
    features: ['label', 'text'],
    num_rows: 8739
})

In [30]:
import os

# Set CUDA_LAUNCH_BLOCKING environment variable to 1
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [31]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset_TEMP,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)


Some weights of RobertaModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/cardiffnlp_twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: ignored

In [29]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 349560
  Num epochs = 1
  Total optimization steps = 21848
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/21848 [00:00<?, ?it/s]

RuntimeError: ignored